## Preparation

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

In [ ]:
# device check(CFDLab)
device=torch.device('cuda:1') #先調1再調0
print(torch.cuda.is_available())
x=torch.randn(100).to(device) # Gives warning

In [ ]:
device=torch.device('cuda:0') # Fix warning
x=torch.randn(100).to(device) # No warning

In [ ]:
# device check(my pc)
device=torch.device('cpu')
print(torch.cuda.is_available())
x=torch.randn(100).to(device) # No warning

## Parameters

In [ ]:
batch=100 #batchsize
opt=1 # optimizer
tol=1e-8 # tolerance
ch=[1, 3, 3, 3, 3, 1] #model layout
pad=[0, 0, 0, 0, 0]
PATH= "DEL_SUP_CNN_TEST" # model name, saved as..

## Data

In [ ]:
# Step 1. Prepare data-1/2
# DataLoader wraps a Dataset and provides minibatches, shuffling, multithreading, for you
data_in=np.loadtxt('./data/preserved/input_div_U_2_1s.dat')
data_out=np.loadtxt('./data/preserved/output_P_1s.dat')
x_in=torch.Tensor(data_in)
y_in=torch.Tensor(data_out)

x_in=x_in[1:]-x_in[:-1]
y_in=y_in[1:]-y_in[:-1]

x_in=x_in.to(device)
y_in=y_in.to(device)

In [ ]:
x=x_in
y=y_in
print(x.size())
print(y.size())

In [ ]:
loader=DataLoader(TensorDataset(x, y), batch_size=batch)

## Model

In [ ]:
# Convolutional neural network (two convolutional layers)
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(torch.nn.Module):
    def __init__(self, channel_1, channel_2, channel_3, channel_4, channel_5, kernel_dim):
        super(ConvNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, channel_1, kernel_dim, padding=pad[0])
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_dim, padding=pad[1])
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_dim, padding=pad[2])
        self.conv4 = nn.Conv2d(channel_3, channel_4, kernel_dim, padding=pad[3])
        self.conv5 = nn.Conv2d(channel_4, channel_5, kernel_dim, padding=pad[4])
        # an affine operation: y = Wx + b
        lin_d=80-(pad[0]+pad[1]+pad[2]+pad[3]+pad[4])*2
        self.fc1 = nn.Linear(70*70*channel_5, 6400)  # 78*78 from image dimension
        #self.fc2 = nn.Linear(1000, 1000)
        #self.fc3 = nn.Linear(1000, 6400)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc1(x)
        return x

model = ConvNet(channel_1=ch[1], channel_2=ch[2], channel_3=ch[3], channel_4=ch[4], channel_5=1, kernel_dim=3).to(device)

## Training

In [ ]:
if opt==0:
    optimizer=torch.optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
elif opt==1:
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3)


scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5000, gamma=0.1)

In [ ]:
loss_epoch=[]
loss_values = []
loss=1
epochs=0

print("Epochs    Loss")

while(loss>tol):
    epochs=epochs+1
    scheduler.step()
    
    for x_batch, y_batch in loader:
        # Forward pass
        x_batch= torch.nn.Sequential(torch.nn.Unflatten(1, (1,80,80)))(x_batch)
        y_pred=model(x_batch)        
        loss=torch.nn.functional.mse_loss(y_pred, y_batch)
        
        # Backward pass
        loss.requires_grad_(True)
        loss.backward()
        
        # Update Weights
        optimizer.step()
        optimizer.zero_grad()
        #print("1 batch")
    
    loss_epoch.append(epochs)
    loss_values.append(loss.item())
    
    if epochs%1==0:
        print("Epochs: ", epochs, "; Loss: ", loss.item())
        
    loss=loss.item()

print(epochs, "    ", loss.item())

#Plot loss function
from matplotlib import pyplot as plt
plt.plot(loss_epoch, loss_values)
plt.xlabel('epochs')
plt.ylabel('loss')

## Result

In [ ]:
# Append name with time
from datetime import datetime
ct=datetime.now()
ctime=str(ct.year)+str(ct.month)+str(ct.day)+"_"+str(ct.hour)+str(ct.minute)+str(ct.second)
PATH=PATH+"_"+ctime

In [ ]:
#Plot loss function
from matplotlib import pyplot as plt
plt.plot(loss_epoch, loss_values)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.semilogy()
plt.savefig("./result/"+PATH+'.png')

## Export

In [ ]:
traced_net=torch.jit.trace(model, (torch.randn(1, 1, 80,80)).to(device))
traced_net.to(torch.float64)
torch.jit.save(traced_net, "./result/"+PATH+".pth")

In [ ]:
# Save information
with open("./result/"+PATH+'.txt', 'w') as f:
        f.write('# Model info\n')
        f.write('Model: '+ PATH+'\n')
        f.write('ID: '+ ctime +'\n')
        f.write('Data format: '+ 'Delta' +'\n')
        f.write('Learning method: Supervised'+'\n')
        f.write('Model: CNN'+'\n')
        f.write('residual: '+str(tol)+'\n')
        f.write('batch_size: '+str(batch)+'\n')
        f.write('epochs: '+str(epochs)+'\n')